In [93]:
import math 

def discretize(waiting_times):
    for waiting_time in waiting_times:
        if waiting_time > 99.9:
            waiting_time = 99
    return 10000000* abs(round(waiting_times[0])) + 10000*abs(round(waiting_times[1])) + 10*abs(round(waiting_times[2]))

In [102]:
import gym
from gym import spaces
import numpy as np
from math import ceil

class TrafficGenerator(gym.Env):
    
    #doesnt really workj
    def getObs(self):
        # Loop over the packet types
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
           
            observation.append([current_length, current_waiting_time])

       
        observation = np.array(observation)
        return observation
    
    def __init__(self):
        
        self.action_space = spaces.Discrete(3)
        
        # Define the observation space (number of packets in each queue and their waiting time)
        # Queue maximum length set to 100
        self.observation_space = spaces.Discrete(100000000)
        
        # pancket info (DataType, arrival_rate, mean_delay )
        #self.packetInfo = [[0, 0.3, 6],[1, 0.25, 4],[2, 0.4, float('inf')]]
        
        self.dataType = [0, 1, 2]
        self.arrival_rate = [0.3, 0.25, 0.4]
        self.mean_delay_req = [6, 4, float('inf')]
        self.packetInfo = [[elem1, elem2, elem3] for elem1, elem2, elem3 in zip(self.dataType, self.arrival_rate, self.mean_delay_req)]
        self.curr_mean_delay_best_effort = 0
        self.packet = 1
        self.timeslot = 1
        self.totaltime = self.timeslot
        
        # Initialize the queues
        self.queues = [[], [], []]
      
    
    def step(self, action):  
        self.totaltime += self.timeslot
        print(self.totaltime)
        #packet generator
        for i in range(len(self.packetInfo)):
            for sublist in self.packetInfo:
                if sublist[0] == i and np.random.uniform() < sublist[1]:
                    self.queues[i].append(self.packet)
                    #print("Appending to queue", i)
        
        for i in range(len(self.queues)):
            for j in range(len(self.queues[i])):
                self.queues[i][j] -= self.timeslot
                
        
        # services the queues
        #for i in range(len(self.queues)):
            #print("queue ", i, ": has a length of", len(self.queues[i]))
        #print("ACTION queue ", action, ": has a length of", len(self.queues[action]))
        if action < 3 and len(self.queues[action]) > 0:
            self.queues[action].pop(0)
            for i in range(len(self.queues[action])):
                self.queues[action][i] -= 1
        

        
        #TODO rewardoptions?
        reward = 1
        
        observation = []
        waiting_times = []
        for i in range(len(self.mean_delay_req)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            current_waiting_time = np.round(current_waiting_time, 1)
            observation.append([current_length, current_waiting_time])
            waiting_times.append(current_waiting_time)
            if i == 2:
                #reward option
                if self.curr_mean_delay_best_effort == 0:
                    curr_mean_delay_best_effort = current_waiting_time
                if self.curr_mean_delay_best_effort < current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = 1
                elif self.curr_mean_delay_best_effort > current_waiting_time:
                    self.curr_mean_delay_best_effort = current_waiting_time
                    reward = -1
                else:
                    reward = 1
        
                #print(self.curr_mean_delay_best_effort)
                #print(current_waiting_time)
        print(f"observation{observation}")
        if observation[0][1] < -4:
            reward = -10
        elif observation[1][1] < -6:
            reward = -10
        observation = np.array(observation)
        
         # Loop over the packet types
        """
        observation = []
        for i in range(len(self.mean_delay)):

            # Get the current queue for the packet type
            current_queue = self.queues[i]

            # Calculate the length and average waiting time of the current queue
            current_length = len(current_queue)
            current_waiting_time = np.average(current_queue) if current_length > 0 else 0.0
            #print(np.average(current_queue), current_length, current_waiting_time )
            
            
    
            observation.append([current_length, current_waiting_time])

        observation = np.array(observation)
        """
        info = {"mean_delay_0": self.mean_delay_req[0],
        "mean_delay_1": self.mean_delay_req[1],
        "mean_delay_2": self.mean_delay_req[2],
        "arrival_rate_0": self.arrival_rate[0],
        "arrival_rate_1": self.arrival_rate[1],
        "arrival_rate_2": self.arrival_rate[2]}
        done = len(self.queues[0]) + len(self.queues[1]) + len(self.queues[2]) == 0 or (self.totaltime >= 100)
        
        waiting_times = discretize(waiting_times)
        
        return waiting_times, reward, done, info
        
    def reset(self):
        # Reset the queues
        self.queues = [[], [], []]
        self.totaltime = 1
        return [0,0,0]
        
    def render(self):
        pass

In [103]:
env = TrafficGenerator()
print('State space: ', env.observation_space)
print('Action space: ', env.action_space)


State space:  Discrete(100000000)
Action space:  Discrete(3)


In [104]:
env = TrafficGenerator()
obs = env.reset()
done = False
x = 0
while x < 100:
    print("~~~~~~~~~")
    action = env.action_space.sample()
    
    print("chosen", action)
    obs,reward, done, info = env.step(action)
    print(obs)
    print("reward", reward)
    
    x = x +1

~~~~~~~~~
chosen 2
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
0
reward 1
~~~~~~~~~
chosen 1
3
observation[[0, 0.0], [0, 0.0], [1, 0.0]]
0
reward 1
~~~~~~~~~
chosen 2
4
observation[[1, 0.0], [1, 0.0], [1, -1.0]]
1
reward -1
~~~~~~~~~
chosen 1
5
observation[[1, -1.0], [1, -1.0], [2, -1.0]]
10101
reward 1
~~~~~~~~~
chosen 1
6
observation[[2, -1.0], [0, 0.0], [3, -1.3]]
10001
reward -1
~~~~~~~~~
chosen 1
7
observation[[2, -2.0], [0, 0.0], [4, -1.8]]
20002
reward -1
~~~~~~~~~
chosen 0
8
observation[[1, -3.0], [0, 0.0], [4, -2.8]]
30003
reward -1
~~~~~~~~~
chosen 0
9
observation[[0, 0.0], [0, 0.0], [4, -3.8]]
4
reward -1
~~~~~~~~~
chosen 0
10
observation[[0, 0.0], [0, 0.0], [4, -4.8]]
5
reward -1
~~~~~~~~~
chosen 2
11
observation[[0, 0.0], [0, 0.0], [4, -4.8]]
5
reward 1
~~~~~~~~~
chosen 0
12
observation[[0, 0.0], [0, 0.0], [5, -4.6]]
5
reward 1
~~~~~~~~~
chosen 0
13
observation[[0, 0.0], [1, 0.0], [5, -5.6]]
6
reward -1
~~~~~~~~~
chosen 0
14
observation[[0, 0.0], [1, -1.0], [6, -5.5]]
106


In [106]:
import random 

action_size = env.action_space.n
state_size = env.observation_space.n
qtable = np.zeros((state_size, action_size))

# Hyperparameters
total_episodes = 500
total_test_episodes = 10
max_steps = 99
learning_rate = 0.7
gamma = 0.618
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

# Train
for episode in range(total_episodes):
    state = env.reset()
    
    for step in range(max_steps):
        exp_exp_tradeoff = random.uniform(0, 1)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state, :])
        else:
            action = env.action_space.sample()

        new_state, reward, done, info = env.step(action)
        new_state = int((new_state)*10)
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state]) - qtable[state, action])
        
        state = new_state
        if done: break

    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * (episode+1))


# Play the Game
rewards = []

for episode in range(total_test_episodes):
    total_rewards = 0
    total_penalties = 0
    state = env.reset()
    print('='*20)
    print("[*] Episode", episode)
    print('='*20)

    for step in range(max_steps):
        env.render()
        action = np.argmax(qtable[state, :])
        state, reward, done, info = env.step(action)
#         print(reward)
        if reward == -10:
            penlties += 1
        else:
            total_rewards += reward
        print("total time: {}", env.totaltime)
        if done:
            rewards.append(total_rewards)
            print('[*] Score', total_rewards)
            break

env.close()
print('[*] Average Score: ' + str(sum(rewards) / total_test_episodes))

2
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
3
observation[[1, -1.0], [0, 0.0], [1, 0.0]]
4
observation[[1, -2.0], [0, 0.0], [1, -1.0]]
5
observation[[1, -3.0], [0, 0.0], [1, -2.0]]
6
observation[[2, -2.0], [0, 0.0], [1, -1.0]]
7
observation[[2, -3.0], [0, 0.0], [1, -2.0]]
8
observation[[1, -3.0], [0, 0.0], [2, -1.5]]
9
observation[[1, -1.0], [0, 0.0], [3, -1.7]]
10
observation[[0, 0.0], [0, 0.0], [3, -2.7]]
11
observation[[0, 0.0], [0, 0.0], [4, -2.8]]
12
observation[[0, 0.0], [1, 0.0], [5, -3.0]]
13
observation[[0, 0.0], [1, -1.0], [5, -4.0]]
14
observation[[0, 0.0], [1, -2.0], [6, -4.2]]
15
observation[[1, 0.0], [1, -1.0], [6, -5.2]]
16
observation[[1, -1.0], [1, -2.0], [5, -6.2]]
17
observation[[1, -2.0], [0, 0.0], [5, -7.2]]
18
observation[[1, -3.0], [0, 0.0], [6, -6.8]]
19
observation[[1, -1.0], [0, 0.0], [7, -6.7]]
20
observation[[2, -1.0], [0, 0.0], [8, -6.8]]
21
observation[[2, -1.5], [0, 0.0], [8, -7.8]]
22
observation[[3, -1.7], [0, 0.0], [8, -8.8]]
23
observation[[4, -2.0], 

2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[1, 0.0], [0, 0.0], [0, 0.0]]
3
observation[[1, -1.0], [0, 0.0], [0, 0.0]]
4
observation[[2, -1.0], [0, 0.0], [0, 0.0]]
5
observation[[2, -2.0], [0, 0.0], [1, 0.0]]
6
observation[[2, -3.0], [0, 0.0], [0, 0.0]]
7
observation[[3, -2.7], [0, 0.0], [0, 0.0]]
8
observation[[3, -3.7], [0, 0.0], [0, 0.0]]
9
observation[[3, -3.3], [0, 0.0], [0, 0.0]]
10
observation[[3, -3.0], [0, 0.0], [0, 0.0]]
11
observation[[2, -4.0], [0, 0.0], [0, 0.0]]
12
observation[[1, -5.0], [0, 0.0], [0, 0.0]]
13
observation[[1, -1.0], [0, 0.0], [0, 0.0]]
14
observation[[2, -1.0], [1, 0.0], [0, 0.0]]
15
observation[[2, -2.0], [2, -0.5], [0, 0.0]]
16
observation[[2, -3.0], [2, -1.5], [0, 0.0]]
17
observation[[1, -4.0], [2, -2.5], [0, 0.0]]
18
observation[[1, -1.0], [2, -3.5], [0, 0.0]]
19
observation[[2, -1.0], [1, -5.0], [1, 0.0]]
20
observation[[1, -2.0], [2, -3.0], [2, -0.5]]
21
observation[[1, -3.0], [2, -1.5], [3, -1.0]]
22
observation[[1, -1.0], [2, -2.5],

83
observation[[0, 0.0], [1, -1.0], [12, -15.5]]
84
observation[[0, 0.0], [1, -2.0], [12, -16.5]]
85
observation[[0, 0.0], [1, -3.0], [12, -17.5]]
86
observation[[1, 0.0], [0, 0.0], [13, -17.1]]
87
observation[[1, -1.0], [0, 0.0], [13, -18.1]]
88
observation[[0, 0.0], [1, 0.0], [13, -19.1]]
89
observation[[0, 0.0], [0, 0.0], [13, -20.1]]
90
observation[[1, 0.0], [0, 0.0], [13, -19.2]]
91
observation[[1, -1.0], [1, 0.0], [12, -19.8]]
92
observation[[1, -2.0], [2, -0.5], [12, -18.8]]
93
observation[[1, -3.0], [1, -2.0], [13, -18.3]]
94
observation[[0, 0.0], [1, -3.0], [13, -19.3]]
95
observation[[0, 0.0], [1, -1.0], [13, -20.3]]
96
observation[[0, 0.0], [1, -2.0], [13, -21.3]]
97
observation[[0, 0.0], [2, -1.5], [14, -20.7]]
98
observation[[0, 0.0], [2, -2.5], [14, -21.7]]
99
observation[[0, 0.0], [3, -2.3], [14, -22.7]]
100
observation[[1, 0.0], [2, -3.0], [15, -22.1]]
2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[0, 0.0], [1, 0.0], [1, 0.0]]
3
observation[[0, 0.0], [1, -1.

3
observation[[0, 0.0], [1, -1.0], [0, 0.0]]
4
observation[[0, 0.0], [1, -2.0], [0, 0.0]]
5
observation[[1, 0.0], [1, -3.0], [0, 0.0]]
6
observation[[1, -1.0], [2, -2.0], [0, 0.0]]
7
observation[[1, -2.0], [1, -2.0], [0, 0.0]]
8
observation[[0, 0.0], [1, -3.0], [0, 0.0]]
9
observation[[0, 0.0], [1, -4.0], [0, 0.0]]
10
observation[[0, 0.0], [1, -5.0], [0, 0.0]]
11
observation[[0, 0.0], [1, -6.0], [0, 0.0]]
12
observation[[0, 0.0], [2, -3.5], [1, 0.0]]
13
observation[[0, 0.0], [1, -2.0], [1, -1.0]]
14
observation[[0, 0.0], [1, -1.0], [1, -2.0]]
15
observation[[0, 0.0], [1, -2.0], [1, -3.0]]
16
observation[[0, 0.0], [1, -3.0], [0, 0.0]]
17
observation[[1, 0.0], [1, -4.0], [0, 0.0]]
18
observation[[0, 0.0], [1, -5.0], [0, 0.0]]
19
observation[[0, 0.0], [1, -6.0], [0, 0.0]]
20
observation[[0, 0.0], [2, -3.5], [0, 0.0]]
21
observation[[0, 0.0], [2, -4.5], [0, 0.0]]
22
observation[[0, 0.0], [2, -5.5], [1, 0.0]]
23
observation[[0, 0.0], [2, -6.5], [1, -1.0]]
24
observation[[0, 0.0], [2, -3.0],

29
observation[[3, -6.3], [3, -5.3], [3, -2.0]]
30
observation[[3, -7.3], [3, -4.7], [3, -3.0]]
31
observation[[2, -6.0], [3, -5.7], [4, -3.0]]
32
observation[[2, -7.0], [2, -6.0], [5, -3.2]]
33
observation[[1, -7.0], [3, -4.7], [6, -3.5]]
34
observation[[1, -1.0], [4, -4.2], [6, -4.5]]
35
observation[[2, -1.0], [4, -5.2], [5, -5.6]]
36
observation[[2, -2.0], [3, -5.3], [6, -5.5]]
37
observation[[2, -3.0], [3, -6.3], [5, -6.6]]
38
observation[[1, -4.0], [3, -7.3], [6, -6.3]]
39
observation[[1, -5.0], [3, -5.3], [6, -7.3]]
40
observation[[2, -3.0], [2, -6.0], [7, -7.1]]
41
observation[[3, -2.7], [1, -5.0], [7, -8.1]]
42
observation[[2, -2.5], [1, -6.0], [7, -9.1]]
43
observation[[3, -2.3], [1, -7.0], [6, -10.2]]
44
observation[[3, -3.3], [1, -1.0], [6, -11.2]]
45
observation[[3, -4.3], [2, -1.0], [5, -12.2]]
46
observation[[3, -4.0], [2, -2.0], [6, -11.0]]
47
observation[[3, -5.0], [1, -3.0], [6, -12.0]]
48
observation[[3, -6.0], [2, -2.0], [5, -12.6]]
49
observation[[4, -5.2], [2, -1.5

2
observation[[0, 0.0], [0, 0.0], [0, 0.0]]
2
observation[[1, 0.0], [1, 0.0], [0, 0.0]]
3
observation[[1, -1.0], [1, -1.0], [0, 0.0]]
4
observation[[1, -2.0], [1, -2.0], [0, 0.0]]
5
observation[[1, -1.0], [1, -3.0], [0, 0.0]]
6
observation[[1, -2.0], [1, -4.0], [0, 0.0]]
7
observation[[1, -1.0], [2, -2.5], [0, 0.0]]
8
observation[[0, 0.0], [2, -3.5], [0, 0.0]]
9
observation[[0, 0.0], [2, -2.0], [0, 0.0]]
10
observation[[0, 0.0], [2, -2.0], [1, 0.0]]
11
observation[[0, 0.0], [1, -3.0], [2, -0.5]]
12
observation[[1, 0.0], [2, -2.0], [2, -1.5]]
13
observation[[1, -1.0], [3, -2.0], [2, -2.0]]
14
observation[[1, -2.0], [3, -3.0], [1, -3.0]]
15
observation[[1, -3.0], [3, -4.0], [0, 0.0]]
16
observation[[0, 0.0], [3, -5.0], [1, 0.0]]
17
observation[[0, 0.0], [2, -5.5], [2, -0.5]]
18
observation[[0, 0.0], [2, -6.5], [2, -1.5]]
19
observation[[0, 0.0], [1, -8.0], [2, -2.5]]
20
observation[[0, 0.0], [2, -4.5], [2, -3.5]]
21
observation[[0, 0.0], [3, -3.7], [3, -3.0]]
22
observation[[1, 0.0], [4,

NameError: name 'penlties' is not defined